In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
import os
from concurrent.futures import ThreadPoolExecutor

class VideoCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Video Cropper")

        # Se inicializa el canvas sin tamaño específico, se ajustará al tamaño del video
        self.canvas = tk.Canvas(root)
        self.canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.lines = []
        self.coordinates = []
        self.video_files = []
        self.current_index = 0

        # Configura los comandos de teclado
        self.root.bind("<Key>", self.on_key_press)
    
    def on_key_press(self, event):
        # Verifica el carácter en minúscula para manejar las teclas
        if event.char.lower() == 'l':
            self.load_videos()
        elif event.char.lower() == 'n':
            self.next_video()
        elif event.char.lower() == 'c':
            self.crop_videos()
    
    def load_videos(self):
        folderpath = filedialog.askdirectory()
        if folderpath:
            self.video_files = [os.path.join(folderpath, f) for f in os.listdir(folderpath) if f.endswith(('.mp4', '.avi', '.mov'))]
            if self.video_files:
                self.current_index = 0
                self.show_video(self.current_index)
            else:
                messagebox.showinfo("No Videos Found", "No video files found in the selected directory.")

    def show_video(self, index):
        self.filepath = self.video_files[index]
        video_clip = VideoFileClip(self.filepath)
        frame = video_clip.get_frame(0)
        self.image = Image.fromarray(frame)

        self.canvas.config(width=video_clip.size[0], height=video_clip.size[1])
        self.photo = ImageTk.PhotoImage(self.image)
        self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)

        video_clip.close()
        self.canvas.bind("<Button-1>", self.on_canvas_click)

    def next_video(self):
        if self.current_index + 1 < len(self.video_files):
            self.current_index += 1
            self.lines = []
            self.coordinates = []
            self.canvas.delete("all")
            self.show_video(self.current_index)
        else:
            if not self.coordinates:
                messagebox.showinfo("End of List", "No more videos to mark.")



    def on_canvas_click(self, event):
        if len(self.coordinates) < 3:
            video_clip = VideoFileClip(self.filepath)
            video_width, video_height = video_clip.size
            line = self.canvas.create_line(event.x, 0, event.x, video_height, fill="red", width=2)
            self.coordinates.append(event.x)
            print(event.x)


    def crop_videos(self):
        if not self.video_files or not self.coordinates:
            messagebox.showerror("Error", "No videos to crop or crop lines not set.")
            return
        output_folder = filedialog.askdirectory()
        if output_folder and self.coordinates:
            with ThreadPoolExecutor() as executor:
                for filepath in self.video_files:
                    executor.submit(self.crop_video, filepath, self.coordinates, output_folder)

    def crop_video(self, filepath, coordinates, output_folder):
        if len(coordinates) == 3:
            col1, col2, col3 = sorted(coordinates)
            
            print(col1, col2, col3)

            video_clip = VideoFileClip(filepath)

            video1 = crop(video_clip, x1=col1, y1=0, x2=col2, y2=video_clip.size[1])
            video1_output_path = os.path.join(output_folder, os.path.basename(filepath).split('.')[0] + "_CS.mp4")
            video1.write_videofile(video1_output_path, codec='libx264', audio_codec='aac', verbose = False)

            video2 = crop(video_clip, x1=col2, y1=0, x2=col3, y2=video_clip.size[1])
            video2_output_path = os.path.join(output_folder, os.path.basename(filepath).split('.')[0] + "_P.mp4")
            video2.write_videofile(video2_output_path, codec='libx264', audio_codec='aac', verbose = False)

            video_clip.close()
            video1.close()
            video2.close()

if __name__ == "__main__":
    root = tk.Tk()
    app = VideoCropper(root)
    root.mainloop()
